In [5]:
"""
Model of circadian oscillator with entrainment
"""
import numpy as np
import matplotlib as plt
import pandas as pd
from scipy.integrate import odeint
from pylab import *
from scipy import signal
from scipy.signal import argrelextrema
%matplotlib inline

#---------------------------------------------------------------------------------------------
#    Increasing hill function
#---------------------------------------------------------------------------------------------    
def hill_up(x, k, n):
    h = np.power(x,n)/(np.power(k,n) + np.power(x,n))
    return h

#---------------------------------------------------------------------------------------------
#    Decreasing hill function
#---------------------------------------------------------------------------------------------    
def hill_down(x, k, n):
    h = np.power(k,n)/(np.power(k,n) + np.power(x,n))
    return h

#---------------------------------------------------------------------------------------------
#    Generage a squate function of time series t with given period and amplitude
#---------------------------------------------------------------------------------------------
def square_wave(t, period, amp):
    wave = amp*signal.square(2 * np.pi * t/period)+amp
    return wave
            
#---------------------------------------------------------------------------------------------
#    Function defining differential equation for the model
#---------------------------------------------------------------------------------------------
def df(y, t):
    #assign input vector,y to variables of the model
    M  = y[0]
    PC = y[1]
    PN = y[2]

 
    # mRNA expression
    dM =  vs*hill_down(PN, KI, 4) - vm*hill_up(M, Km, 1) - square_wave(t, light_period, light_amp)*M
    # Protein concentration, PC - celllular, PN - nuclear
    dPC =   ks*M + k2*PN - k1*PC    - vd*hill_up(PC, Kd, 1)
    dPN =        - k2*PN + k1*PC
    # output right-hand side of differetial equations
    return [dM, dPC, dPN]

#---------------------------------------------------------------------------------------------
#    Find location and amplitude of peaks in a variable v with given timeline t
#---------------------------------------------------------------------------------------------
def find_peaks(t,v):
    # convert vontage trace into array
    v_array = np.array(v, dtype=np.float)
    # Find indices of local max(spikes)
    locs_max = argrelextrema(v_array, np.greater, order=2)[0]
    # Record spike times and max voltages in a dataframe
    peak_times = t[locs_max]
    peak_amp   = v[locs_max]
    return(peak_times, peak_amp)

#---------------------------------------------------------------------------------------------
#    Plot all variable of the model for the last plotting_time hours
#    Each plot is labeled by information in list names
#---------------------------------------------------------------------------------------------
def periods_list(soln,dt, names, non_transient_time):
    t  = np.arange(0,non_transient_time, dt)
    n_ind = len(t)
    n_vars = len(variable_names)
    periods = []
    for i in range(0,n_vars):
        variable = soln[-n_ind:,i]
        [peaks, amp] = find_peaks(t,variable)
        periods.append(peaks[-1]-peaks[-2])
    return periods

#---------------------------------------------------------------------------------------------
#    Parameters of the model
#---------------------------------------------------------------------------------------------
vs = 1.6; vm = 0.505; Km = 0.5
KI = 1.0; ks = 0.2; vd = 1.4
Kd = 0.13; k1 = 0.5; k2 = 0.6

#---------------------------------------------------------------------------------------------
# Parameters for light stimulus (amplitude 0 means DD)
#---------------------------------------------------------------------------------------------
light_period = 24
light_amp = 0

#---------------------------------------------------------------------------------------------
#   Initial conditions
#---------------------------------------------------------------------------------------------
y0= [2.15404051, 3.19312247, 3.11948775] 

# List with variable names
variable_names = ['mRNA', 'Cytosol protein', 'Nuclear Protein']

#---------------------------------------------------------------------------------------------
#   Run the simulation of the model 
#---------------------------------------------------------------------------------------------
# Choose total time of simulation, T
T = 1000.
# Chose time step
dt=0.05
# Make a time grid for the simulation
nsteps=int(T/dt)
t  = np.linspace(0, T, nsteps) 
# Chose number of simulation hours(last) which will be the data portion
non_transient_T = 100
vs_list = [1.0,1.6]
ks_list = [0.11 + 0.01*i for i in range(0,90)]
#ks_list = [0.11 + 0.1*i for i in range(0,10)]
for vs in vs_list:
    period_list = []
    print(ks_list)
    for ks in ks_list:
        # Solve ODE for the model and plot the solution
        soln = odeint(df, y0, t)
        periods_ret = periods_list(soln, dt, variable_names, non_transient_T)
        period_avg = np.array(periods_ret).mean()
        print('vs = ',vs,' ks = ',ks,': period_avg = ',period_avg)
        period_list.append(period_avg)
    
    dfout = pd.DataFrame(list(zip(ks_list, period_list)), columns =['ks', 'period'])
    filename = 'ks_vs_periods_vs=' + str(vs) + '.dat'
    dfout.to_csv(filename,index=False)

[0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.16999999999999998, 0.18, 0.19, 0.2, 0.21000000000000002, 0.22, 0.22999999999999998, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.39999999999999997, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.5700000000000001, 0.5800000000000001, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.9400000000000001, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]


<ipython-input-5-bcaf5bd38136>:57: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v_array = np.array(v, dtype=np.float)


vs =  1.0  ks =  0.11 : period_avg =  29.433333333333337
vs =  1.0  ks =  0.12 : period_avg =  28.233333333333334
vs =  1.0  ks =  0.13 : period_avg =  27.200000000000003
vs =  1.0  ks =  0.14 : period_avg =  26.26666666666667
vs =  1.0  ks =  0.15 : period_avg =  25.516666666666666
vs =  1.0  ks =  0.16 : period_avg =  24.983333333333334
vs =  1.0  ks =  0.16999999999999998 : period_avg =  24.666666666666668
vs =  1.0  ks =  0.18 : period_avg =  24.433333333333337
vs =  1.0  ks =  0.19 : period_avg =  24.2
vs =  1.0  ks =  0.2 : period_avg =  23.96666666666667
vs =  1.0  ks =  0.21000000000000002 : period_avg =  23.73333333333333
vs =  1.0  ks =  0.22 : period_avg =  23.533333333333335
vs =  1.0  ks =  0.22999999999999998 : period_avg =  23.3
vs =  1.0  ks =  0.24 : period_avg =  23.083333333333332
vs =  1.0  ks =  0.25 : period_avg =  22.850000000000005
vs =  1.0  ks =  0.26 : period_avg =  22.650000000000002
vs =  1.0  ks =  0.27 : period_avg =  22.433333333333337
vs =  1.0  ks =  0

vs =  1.6  ks =  0.55 : period_avg =  20.816666666666677
vs =  1.6  ks =  0.56 : period_avg =  20.683333333333337
vs =  1.6  ks =  0.5700000000000001 : period_avg =  20.55
vs =  1.6  ks =  0.5800000000000001 : period_avg =  20.41666666666667
vs =  1.6  ks =  0.59 : period_avg =  20.3
vs =  1.6  ks =  0.6 : period_avg =  20.183333333333334
vs =  1.6  ks =  0.61 : period_avg =  20.066666666666674
vs =  1.6  ks =  0.62 : period_avg =  19.950000000000003
vs =  1.6  ks =  0.63 : period_avg =  19.85000000000001
vs =  1.6  ks =  0.64 : period_avg =  19.733333333333334
vs =  1.6  ks =  0.65 : period_avg =  19.63333333333334
vs =  1.6  ks =  0.66 : period_avg =  19.533333333333342
vs =  1.6  ks =  0.67 : period_avg =  19.41666666666667
vs =  1.6  ks =  0.68 : period_avg =  19.300000000000004
vs =  1.6  ks =  0.69 : period_avg =  19.200000000000003
vs =  1.6  ks =  0.7 : period_avg =  19.13333333333333
vs =  1.6  ks =  0.71 : period_avg =  19.01666666666667
vs =  1.6  ks =  0.72 : period_avg =  